In [1]:
import os
import pandas as pd
import numpy as np
from config import Config

In [2]:
!pip install xlrd


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def fileDataframe(fichier, parametre):
    data = pd.read_excel(fichier)
    data_trans =data.melt(id_vars=['Station','Annee','Mois'], value_vars= data.columns[3:],var_name='Jour',value_name=parametre)
    nom={
'Bobo-Dioulasso':'Bobo_Dioulasso', 'Bogandé':'Bogande', 'Boromo':'Boromo', 'Dédougou':'Dedougou', 'Dori':'Dori', 
'Fada Ngourma':'Fada_Ngourma', 'Gaoua':'Gaoua', 'Ouagadougou aéro':'Ouagadougou_aero', 'Ouahigouya':'Ouahigouya', 'Pô':'Po'}
    data_trans['Station']=data_trans['Station'].replace(nom)
    return data_trans

In [4]:
#Creer une time serie en fusionnant l'Annee, le mois et le Jour pour creer un index de date
def convertTimeSerie(df):
    # Fonction de validation pour vérifier si une date est valide
    def dateValid(year, month, day):
        try:
            pd.Timestamp(year=year, month=month, day=day)
            return True
        except ValueError:
            return False 
    # Créer une nouvelle colonne "Date" en fusionnant les colonnes "Jour", "Mois" et "Annee"
    df['Date'] = df.apply(lambda row: str(row['Jour']) + '/' + str(row['Mois']) + '/' + str(row['Annee']), axis=1)
    # Convertir la colonne "Date" en objet datetime en appliquant la validation
    df.loc[:, 'Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
    # Supprimer les lignes avec des dates non valides
    df = df.dropna(subset=['Date'])
     # Supprimer les colonnes "Annee", "Mois" et "Jour" si nécessaire
    df.drop(['Annee', 'Mois', 'Jour'], axis=1, inplace=True)
    return df

In [5]:
# Lecture du fichier de données journalières de pluie
#pluieData=fileDataframe(str(Config.PLUIE_ETP_PATH)+'/'+str(Config.PLUIE_FILE_NAME),'Pluie')
pluieData=fileDataframe(os.path.join(Config.PLUIE_ETP_PATH,Config.PLUIE_FILE_NAME),'Pluie')
pluieData.head(3)

,Station,Annee,Mois,Jour,Pluie
0,Bobo_Dioulasso,1961,1,01,0.0
1,Bobo_Dioulasso,1961,2,01,0.0
2,Bobo_Dioulasso,1961,3,01,0.0


In [6]:
# Lecture du fichier de données journalières d'evapotranspiration
#petData=fileDataframe(str(Config.PLUIE_ETP_PATH)+'/'+str(Config.ETP_FILE_NAME),'Etp')
petData=fileDataframe(os.path.join(Config.PLUIE_ETP_PATH,Config.ETP_FILE_NAME),'Etp')
petData.head(3)

,Station,Annee,Mois,Jour,Etp
0,Bobo_Dioulasso,1961,1,01,0.0
1,Bobo_Dioulasso,1961,2,01,0.0
2,Bobo_Dioulasso,1961,3,01,0.0


In [7]:
petSerie=convertTimeSerie(petData)
petSerie.head(3)

<positron-console-cell-7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Station,Etp,Date
0,Bobo_Dioulasso,0.0,1961-01-01 00:00:00
1,Bobo_Dioulasso,0.0,1961-02-01 00:00:00
2,Bobo_Dioulasso,0.0,1961-03-01 00:00:00


In [8]:
pluieSerie=convertTimeSerie(pluieData)
pluieSerie.head()

<positron-console-cell-8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Station,Pluie,Date
0,Bobo_Dioulasso,0.0,1961-01-01 00:00:00
1,Bobo_Dioulasso,0.0,1961-02-01 00:00:00
2,Bobo_Dioulasso,0.0,1961-03-01 00:00:00
3,Bobo_Dioulasso,0.0,1961-04-01 00:00:00
4,Bobo_Dioulasso,0.0,1961-05-01 00:00:00


In [9]:
# Fusionner les dataframes en utilisant les colonnes "Date" et "Station" comme clés de fusion
merged_data = pd.merge(pluieSerie, petSerie, on=['Date', 'Station'], how='outer')

merged_data.to_csv(os.path.join(Config.FILES_TRAITED_PATH,Config.PLUIE_ETP_FILE_NAME), index=False)
# Afficher le dataframe fusionné
merged_data.head(3)

,Station,Pluie,Date,Etp
0,Bobo_Dioulasso,0.0,1961-01-01 00:00:00,0.0
1,Bogande,0.0,1961-01-01 00:00:00,0.0
2,Boromo,0.0,1961-01-01 00:00:00,0.0


In [10]:
df_boromo=merged_data[merged_data['Station']=='Boromo']
df_boromo.to_csv(os.path.join(Config.FILES_TRAITED_PATH,Config.STATION_TEST))

In [11]:
print(len(merged_data))
print(len(pluieSerie))
print(len(petSerie))

228520
228520
228520
228520
228520
228520


In [12]:
merged_data.head()

,Station,Pluie,Date,Etp
0,Bobo_Dioulasso,0.0,1961-01-01 00:00:00,0.0
1,Bogande,0.0,1961-01-01 00:00:00,0.0
2,Boromo,0.0,1961-01-01 00:00:00,0.0
3,Dedougou,0.0,1961-01-01 00:00:00,0.0
4,Dori,0.0,1961-01-01 00:00:00,0.0
